In [4]:
import os
import shutil
import subprocess as sp
import time

import numpy as np
from pymatgen.core.structure import Structure

cwd_dir = '/home/lonya/doc/dptb/CrI3'
pseudo_dir = os.path.join(cwd_dir, 'pseudo')
orbital_dir = os.path.join(cwd_dir, 'orbital')
scf_dir = os.path.join(cwd_dir, 'scf')
nscf_dir = os.path.join(cwd_dir, 'nscf')
data_dir = os.path.join(cwd_dir, 'data')

os.makedirs(scf_dir, exist_ok=True)
os.makedirs(nscf_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
soc_list = ["nsoc", "soc"]
for str_soc in soc_list:
    os.makedirs(os.path.join(data_dir, str_soc), exist_ok=True)
    os.makedirs(os.path.join(data_dir, str_soc, "set.0"), exist_ok=True)

In [165]:
np.random.seed(42)

os.chdir(cwd_dir)

structure = Structure.from_file('CrI3.cif')

N_structure = len(structure)

pseudo_dir = os.path.join(cwd_dir, 'pseudo')
orbital_dir = os.path.join(cwd_dir, 'orbital')

os.chdir(scf_dir)

cart_coords = structure.cart_coords
numbers = structure.atomic_numbers
NUM_ATOM = len(numbers)
lattice = structure.lattice.matrix
frac_coords = structure.frac_coords

Cr_frac_coords_str = ''
I_frac_coords_str = ''
for i in range(NUM_ATOM):
    if frac_coords[i, 2] > 0.24 and frac_coords[i, 2] < 0.41:
        if numbers[i] == 24:
            Cr_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2] + 0.166666:.16f} 0 0 0\n'
        elif numbers[i] == 53:
            I_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2] + 0.166666:.16f} 0 0 0\n'
        else:
            print(numbers[i])
            raise ValueError("numbers[i] error")

for int_soc in [0, 1]:
    work_dir = os.path.join(scf_dir, soc_list[int_soc])
    os.makedirs(work_dir, exist_ok=True)
    structure.to('poscar', 'POSCAR')

    in_file = fr"""ATOMIC_SPECIES
I 126.905 I_ONCV_PBE_FR-1.1.upf
Cr 51.996 Cr_ONCV_PBE_FR-1.0.upf

NUMERICAL_ORBITAL
I_gga_6au_100Ry_2s2p2d1f.orb
Cr_gga_7au_100Ry_4s2p2d1f.orb

LATTICE_CONSTANT
1.8897259886 # 1.8897259886 Bohr = 1.0 Angstrom

LATTICE_VECTORS
{lattice[0, 0]:.16f} {lattice[0, 1]:.16f} {lattice[0, 2]:.16f}
{lattice[1, 0]:.16f} {lattice[1, 1]:.16f} {lattice[1, 2]:.16f}
{lattice[2, 0]:.16f} {lattice[2, 1]:.16f} {lattice[2, 2]:.16f}

ATOMIC_POSITIONS
Direct

I
0.0
6
{I_frac_coords_str}

Cr
3.43
2
{Cr_frac_coords_str}
"""
    with open(os.path.join(work_dir, 'STRU'), 'w') as f:
        f.write(in_file)

    in_file = fr"""K_POINTS
0
Gamma
3 3 3 0 0 0
"""
    with open(os.path.join(work_dir, 'KPT'), 'w') as f:
        f.write(in_file)
        
    in_file = fr"""INPUT_PARAMETERS
pseudo_dir              {pseudo_dir}
orbital_dir             {orbital_dir}
ntype                   2
noncolin                1
lspinorb                {int_soc}
calculation             scf
basis_type              lcao
ecutwfc                 100
scf_thr                 1.0e-7
scf_nmax                100
gamma_only              0

ks_solver               genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

out_chg                 1
symmetry                0
"""
    with open(os.path.join(work_dir, 'INPUT'), 'w') as f:
        f.write(in_file)

/root/miniconda3/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [166]:
os.environ['OMP_NUM_THREADS'] = '8'

for str_soc in soc_list:
    work_dir = os.path.join(scf_dir, str_soc)
    os.chdir(work_dir)
    sp.run(['mpirun', '-np', '1', 'abacus'])
    #sp.run(['mpirun', '-np', '1', 'abacus'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Sun May 19 01:38:42 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional electrons can yield (more) accurate outcomes, but may be less efficient.
 If you're confident th

In [5]:
os.chdir(nscf_dir)
for int_soc in [0, 1]:
    work_dir = os.path.join(nscf_dir, soc_list[int_soc])
    os.makedirs(work_dir, exist_ok=True)

    os.makedirs(os.path.join(work_dir, 'OUT.ABACUS'), exist_ok=True)
    shutil.copyfile(os.path.join(scf_dir, soc_list[int_soc], 'OUT.ABACUS/SPIN1_CHG.cube'), os.path.join(work_dir, 'OUT.ABACUS/SPIN1_CHG.cube'))
    shutil.copyfile(os.path.join(scf_dir, soc_list[int_soc], 'OUT.ABACUS/SPIN2_CHG.cube'), os.path.join(work_dir, 'OUT.ABACUS/SPIN2_CHG.cube'))
    shutil.copyfile(os.path.join(scf_dir, soc_list[int_soc], 'OUT.ABACUS/SPIN3_CHG.cube'), os.path.join(work_dir, 'OUT.ABACUS/SPIN3_CHG.cube'))
    shutil.copyfile(os.path.join(scf_dir, soc_list[int_soc], 'OUT.ABACUS/SPIN4_CHG.cube'), os.path.join(work_dir, 'OUT.ABACUS/SPIN4_CHG.cube'))
    shutil.copyfile(os.path.join(scf_dir, soc_list[int_soc], 'OUT.ABACUS/istate.info'), os.path.join(work_dir, 'OUT.ABACUS/istate.info'))
    shutil.copyfile(os.path.join(scf_dir, soc_list[int_soc], 'STRU'), os.path.join(work_dir, 'STRU'))

    in_file = fr"""K_POINTS
5
Line
0.0 0.0 0.0 20 // G
0.5 0.0 0.0 20 // A
0.0 0.5 0.5 20 // Y
0.5 0.5 0.5 20 // M
0.0 0.0 0.0 1 // G
"""
    with open(os.path.join(work_dir, 'KPT'), 'w') as f:
        f.write(in_file)

    in_file = fr"""INPUT_PARAMETERS
pseudo_dir              {pseudo_dir}
orbital_dir             {orbital_dir}
ntype                   2
noncolin                1
lspinorb                {int_soc}
calculation             nscf
basis_type              lcao
ecutwfc                 100
scf_thr                 1.0e-7
scf_nmax                100
gamma_only              0
symmetry                0

ks_solver    genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

init_chg                file
out_band                1
"""

    with open(os.path.join(work_dir, 'INPUT'), 'w') as f:
        f.write(in_file)

In [6]:
os.environ['OMP_NUM_THREADS'] = '8'

for str_soc in soc_list:
    work_dir = os.path.join(nscf_dir, str_soc)

    os.chdir(work_dir)
    sp.run(['mpirun', '-np', '1', 'abacus'])
    #sp.run(['mpirun', '-np', '1', 'abacus'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Sun May 19 15:28:56 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional electrons can yield (more) accurate outcomes, but may be less efficient.
 If you're confident th

In [3]:
'''
grep data type
'''
os.chdir("/home/lonya/doc/dptb/git/DeePTB/examples/Sn_soc/data/non_soc/set.0")

from ase.io.trajectory import Trajectory
filepath = '/home/lonya/doc/dptb/git/DeePTB/examples/Sn_soc/data/soc/set.0/xdat.traj'
xdat = Trajectory(filepath, 'r')
print(xdat[-1])

filepath1 = os.path.join(data_dir, "nsoc/set.0/xdat.traj")
xdat1 = Trajectory(filepath1, 'r')

print(xdat1[-1])

print(xdat[-1].get_initial_magnetic_moments())
print(xdat[-1].get_momenta())


Atoms(symbols='Sn2', pbc=True, cell=[[0.0, 3.2445999999999997, 3.2445999999999997], [3.2445999999999997, 0.0, 3.2445999999999997], [3.2445999999999997, 3.2445999999999997, 0.0]], initial_magmoms=..., momenta=..., constraint=[FixCartesian(a=[0], mask=[False, False, False]), FixCartesian(a=[1], mask=[False, False, False])])
Atoms(symbols='I6Cr2', pbc=True, cell=[[6.968548178391757, 0.0, 3.999999712073049e-16], [-3.4842740891958766, 6.034939749983036, 3.999999712073049e-16], [0.0, 0.0, 20.75585485595739]])
[0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]]


In [8]:
for int_soc in [0, 1]:
    nband, emin, jband = 11 * (int_soc + 1), 0, 2
    with open(os.path.join(nscf_dir, soc_list[int_soc], "OUT.ABACUS/BANDS_1.dat"), 'r') as file:
        lines = file.readlines()
    raw_bands = []
    for line in lines:
        values = line.strip().split()
        raw_bands.append([float(value) for value in values])
    while raw_bands[0][jband] < emin:
        jband += 1
    eigenvalues = []
    for band in raw_bands:
        eigenvalue = []
        eigenvalue.append(band[1])
        eigenvalue.extend(band[j] for j in range(jband, jband + nband))
        eigenvalues.append(eigenvalue)
    eigenvalues = np.array(eigenvalues)
    np.save(os.path.join(data_dir, soc_list[int_soc], "set.0/eigenvalues.npy"), eigenvalues)


In [10]:
with open(os.path.join(nscf_dir, "nsoc/OUT.ABACUS/kpoints"), 'r') as file:
    lines = file.readlines()
#nkstot = int(lines[0].strip().split()[3])
raw_kps = []
for line in lines[3:]:
    values = line.strip().split()
    raw_kps.append([float(value) for value in values[1:4]])
kpoints = np.array(raw_kps)
for str_soc in soc_list:
    np.save(os.path.join(data_dir, str_soc, "set.0/kpoints.npy"), kpoints)


In [12]:
from pymatgen.core import Structure
from ase import Atoms
from ase.io import write
import dpdata

dpdata.System(os.path.join(nscf_dir, "nsoc/STRU"), fmt="abacus/stru").to("vasp/poscar", os.path.join(data_dir, 'nsoc/CrI3.vasp'))
shutil.copyfile(os.path.join(data_dir, 'nsoc/CrI3.vasp'), os.path.join(data_dir, 'soc/CrI3.vasp'))

structure = Structure.from_file(os.path.join(data_dir, 'nsoc/CrI3.vasp'))

positions = structure.frac_coords
elements = [site.specie.symbol for site in structure]
lattice = structure.lattice.matrix

atoms = Atoms(symbols=elements, scaled_positions=positions, cell=lattice, pbc=True)
for str_soc in soc_list:
    write(os.path.join(data_dir, str_soc, 'set.0/xdat.traj'), atoms)


In [6]:
os.chdir(os.path.join(data_dir, "nsoc"))
sp.run(['dptb', 'bond', 'CrI3.vasp'])

 Bond Type         1         2         3         4         5         6         7         8         9        10        11        12        13        14        15        16        17        18        19
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
       I-I      3.71      3.72      3.89      3.89      3.89      3.90      3.90      3.99      3.99      4.00      4.29      4.29      4.30      5.47      5.47      5.47      5.48      5.85      5.87
      I-Cr      2.74      2.74      2.74      2.74      4.97      4.98      4.98      4.98      4.98
     Cr-Cr      4.02      4.02



CompletedProcess(args=['dptb', 'bond', 'CrI3.vasp'], returncode=0)

In [9]:
import json

for int_soc in [0, 1]:
    band_max = 11 * (int_soc + 1)
    data = {
    "nframes": 1,
    "natoms": -1,
    "pos_type": "ase",
    "AtomicData_options": {
        "r_max": 6.0,
        "er_max": 5.0,
        "oer_max": 3.0,
        "pbc": True
    },
    "bandinfo": {
        "band_min": 0,
        "band_max": band_max,
        "emin": None,
        "emax": None
    }
}

    with open(os.path.join(data_dir, soc_list[int_soc], 'set.0/info.json'), 'w') as f:
        json.dump(data, f, indent=4)


In [2]:
from dptb.nn.build import build_model
from dptb.postprocess.bandstructure.band import Band
import matplotlib.pyplot as plt
from ase.io import read
from dptb.data import AtomicData, AtomicDataDict
from dptb.nn.nnsk import NNSK
import json
import torch

In [ ]:
os.chdir(data_dir)
model = build_model(checkpoint='./reference/non_soc/v2ckpt/checkpoint/nnsk.ep100.pth')
jdata={
        "kline_type":"abacus",
        "kpath":[[0.0000000000,   0.0000000000,   0.0000000000,   30],   
                 [0.5000000000,   0.0000000000,   0.5000000000,   30],               
                 [0.6250000000,   0.2500000000,   0.6250000000,   30],    
                 [0.3750000000,   0.3750000000,   0.7500000000,   30],     
                 [0.0000000000,   0.0000000000,   0.0000000000,   30],    
                 [0.5000000000,   0.5000000000,   0.5000000000,   30],                
                 [0.5000000000,   0.2500000000,   0.7500000000,   30],               
                 [0.5000000000,   0.0000000000,   0.5000000000,   1 ]
                 ],
        "klabels":['G','X','U','K','G','L','W','X'],
        "nel_atom":{"Sn":4},
        "E_fermi":-8.2887,
        "emin":-12,
        "emax": 8,
        "ref_band":'./data/non_soc/set.0/eigenvalues.npy'
    }
# calculate the band structure
kpath_kwargs = jdata
bcal = Band(model=model, 
            use_gui=True, 
            results_path='./', 
            device=model.device)

stru_data = "./data/non_soc/Sn.vasp"
AtomicData_options = {"r_max": 6.0, "oer_max":3.0, "pbc": True}

eigenstatus = bcal.get_bands(data=stru_data, 
               kpath_kwargs=kpath_kwargs, 
               AtomicData_options=AtomicData_options)


bcal.band_plot(ref_band = kpath_kwargs["ref_band"],
               E_fermi = kpath_kwargs["E_fermi"],
               emin = kpath_kwargs["emin"],
               emax = kpath_kwargs["emax"])